<a href="https://colab.research.google.com/github/fancagi/intro_pulp/blob/main/Gestion_de_operaciones_L2_Guided.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problema de localización de ambulancias


## Objetivo y Prerrequisitos
En este ejemplo, resolveremos un problema de ubicación de amnbulancias en el cual queremos seleccionar puntos de localización para cubrir las necesidades de la mayor cantidad de barrios posibles. Construiremos un modelo de programación entera mixta (MIP) de este problema, implementaremos este modelo en la interfaz de Python de PuLP + Cbc y calcularemos una solución óptima.



## Motivación

El `maximal coverage location problem` es un problema de optimización que busca determinar la ubicación óptima de un conjunto de instalaciones para maximizar la cobertura de un conjunto de demandas o necesidades. 

Este problema se puede aplicar en diferentes campos, como la planificación de servicios públicos, la ubicación de tiendas minoristas o la asignación de recursos en emergencias. En este ejercicio, los estudiantes deben utilizar programación lineal para abordar un problema de ubicación de instalaciones que maximice la cobertura de una serie de puntos de demanda dentro de un área determinada, utilizando un conjunto limitado de recursos. Los estudiantes deberán definir las variables de decisión, las restricciones y la función objetivo, y utilizar un solver de programación lineal para encontrar la solución óptima al problema planteado.

## Descripción del problema

El problema de ubicación de ambulancias con cobertura máxima (maximal coverage location problem) es un problema de optimización que tiene como objetivo encontrar la ubicación óptima para las estaciones de ambulancias para que la mayor cantidad de personas posible pueda ser atendida en el menor tiempo posible. El objetivo es maximizar el número de personas cubiertas  considerando las limitaciones de recursos como la cantidad de ambulancias disponibles y la distancia máxima permitida para ser recorrida para cada una de ellas.

El problema implica encontrar una ubicación óptima para cada estación de ambulancias de manera que se maximice el número de personas que puedan ser atendidas en un tiempo límite determinado. La ubicación óptima de las estaciones de ambulancias debe ser tal que la mayor cantidad de personas posible estén dentro del rango de tiempo máximo permitido y se pueda prestar atención médica en el menor tiempo posible. Este problema puede ser resuelto utilizando técnicas de programación lineal para encontrar la solución óptima en términos de costo y eficiencia.

## Formulación del Modelo


### Conjuntos e Índices

$i \in I$: Índice y conjunto de ubicaciones de demanda (o pacientes).

$j \in J$: Índice y conjunto de ubicaciones candidatas de ambulancias (o bases).


### Parámetros


$d_{i,j} \in \mathbb{R}^+$: Distancia entre la base $j \in J$ y el paciente $i \in I$.

$\theta_{i,j} \in {0,1}$: Si una ambulancia ubicada en $j \in J$ puede atender a un barrio  $i \in I$ o no. Si $\theta_{i,j}=1$, entonces $j \in J$ puede atender a $i \in I$, y $\theta_{i,j}=0$ en caso contrario.


### Variables de Decisión
$y_{j} \in {0, 1 }$: Esta variable es igual a 1 si se ubica una ambulancia en la ubicación candidata $j \in J$; y 0 en caso contrario.

$ x_{i} \in {0, 1 }$: Esta variable es igual a 1 si la ubicación i es cubierta por alguna  ambulancia 

### Función Objetivo
***Cobertura máxima.*** Queremos maximizar la cantidad de barrios que pueden ser atendidos por las ambulancias ubicadas en las bases seleccionadas. Esto se logra maximizando la suma de las variables que indican que un barrio es cubierto o no.

\begin{equation}
\text{Max} \quad Z =  \sum_{i \in I}x_{i} 
\tag{0}
\end{equation}

### Restricciones

***Cobertura***

Para que una ubicación se considere cubierta debe estar dentro del rango de tiempo permitido.

\begin{equation}
 x_{i} \leq \sum_{j \in J} y_{j}\cdot \theta_{i,j}  \quad \forall j \in J
\tag{1}
\end{equation}

***Límite de ambulancias.***

Debemos asegurarnos de que solo se ubiquen tantas ambulancias como disponibilidad se tiene de estas.

\begin{equation}
\sum_{i \in I}x_{i}\leq N 
\tag{2}
\end{equation}

***Naturaleza de las variables.***

\begin{equation}
x_{i} \in {0, 1 }
\tag{3}
\end{equation}

\begin{equation}
y_{j} \in {0, 1 }
\tag{4}
\end{equation}


In [ ]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np

# Determina la distancia euclidiana entre la bodega seleccionalada y los clientes

def compute_distance(loc1, loc2):
    """compute the distance between two locations

    Args:
        loc1 (tuple): location 1
        loc1 (tuple): location 2

    Returns:
        float: distance between loc1 and loc2
    """
    dx = loc1[0] - loc2[0]
    dy = loc1[1] - loc2[1]
    return sqrt(dx*dx + dy*dy)



def create_grid(x_min, x_max, y_min, y_max, n_points):
    """
    Creates a square grid of points between the specified boundaries.
    
    Parameters
    ----------
    x_min : float
        Minimum x value.
    x_max : float
        Maximum x value.
    y_min : float
        Minimum y value.
    y_max : float
        Maximum y value.
    n_points : int
        Number of points in each dimension.
        
    Returns
    -------
    grid : array
        Array of shape (n_points, n_points, 2) containing the grid of points.
    """
    x = np.linspace(x_min, x_max, n_points)
    y = np.linspace(y_min, y_max, n_points)
    grid = np.array(np.meshgrid(x, y)).T.reshape(-1, 2)
    # convert the grid to a list of tuples
    grid = [tuple(x) for x in grid]
    return grid





In [ ]:
from itertools import product
from math import sqrt

import pulp as plp


# tested with PuLP and Python 3.7.0

# Parameters
 
customers = create_grid(0, 3, 0, 3, 10)

facilities = [(0.18, 1.02),
            (0.6, 2.85),
            (1.23, 0.84),
            (0.63, 1.5),
            (1.71, 1.74),
            (0.24, 0.03),
            (1.2, 2.28),
            (0.42, 2.13),
            (0.09, 2.49),
            (1.62, 2.64),
            (2.46, 0.39),
            (1.83, 0.09),
            (0.78, 1.68),
            (2.76, 0.87),
            (0.48, 1.86),
            (1.86, 0.63),
            (1.59, 2.7),
            (2.04, 1.44),
            (0.54, 1.89),
            (0.06, 2.94)]
setup_cost = [1, 4, 1, 4, 3, 4, 5, 3, 5, 1, 5, 1, 1, 2, 1, 4, 5, 2, 4, 1]
max_distance = 0.75
num_ambulances = 5

beta=np.array((len(facilities),len(customers)))

# Vamos a visualizar la instancia que estaremos resolviendo

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Plot the problem configuration (facilities and customers)
fig = go.Figure()
fig.add_trace(go.Scatter(x=[x for x,y in facilities], y=[y for x,y in facilities], mode='markers', name='Facilities'))
fig.add_trace(go.Scatter(x=[x for x,y in customers], y=[y for x,y in customers], mode='markers', name='Hospital'))
# large circle around the facilities
fig.add_trace(go.Scatter(x=[x for x,y in facilities], y=[y for x,y in facilities], mode='markers', name='Facilities', marker=dict(size=250, color='rgba(255, 0, 0, 0.2)')))
fig.update_layout(title='Facilities and hospitals', xaxis_title='x', yaxis_title='y')
# set plot limits
fig.update_xaxes(range=[0, 3])
fig.update_yaxes(range=[0, 3])
fig.show()

### Preprocesamiento
Definimos una función que determina la distancia euclidiana entre cada instalación y los sitios de los clientes. Además, calculamos los parámetros clave requeridos por la formulación del modelo MIP del problema de ubicación de la instalación.

In [ ]:

# computamos algunos parámetros auxiliares que se necesitán para construir el modelo MIP

# set de clientes y bodegas
I = set(range(len(customers)))
J = set(range(len(facilities)))
cartesian_prod = list(product( I,J))

# Computamos los costos de envío

beta = {(c,f): 1 if compute_distance(customers[c], facilities[f]) <= max_distance else 0 for c, f in cartesian_prod}



## Despliegue del modelo

Ahora definimos el modelo MIP para el problema de ubicación de instalaciones, mediante la definición de las variables de decisión, restricciones y función objetivo. A continuación, iniciamos el proceso de optimización y Gurobi encuentra el plan para construir instalaciones que minimiza los costos totales.

In [ ]:
import pulp as plp
from itertools import product

# MIP model formulation
m = plp.LpProblem('ambulance_location', plp.LpMaximize)


### Definir las variables del modelo

In [ ]:
# Decisión binaria indicando que una ubicaciónn es seleccionada
y = plp.LpVariable.dicts('Select', J, lowBound=0, upBound=1, cat='Binary')

# El barrio es cubierto por una ambulancia
x = plp.LpVariable.dicts('covered', I, lowBound=0, upBound=1, cat='Binary')

Establezca una restricción que asegure que un barrio solo se considera cubierto si hay una ambulancia en una ubicación cercana

In [ ]:
for i in I:    
     m+=x[i]<=plp.lpSum([y[j]*beta[i,j] for j in J])


Escriba una restricción que garantice que no se asignan más ambulancias de las que se tienen disponibles.

In [ ]:
m+=plp.lpSum([y[j] for j in J])<=num_ambulances

Escriba la función objetivo del problema:

Maximizar el número de barrios cubiertos con las ambulancias disponibles.

In [ ]:
m += plp.lpSum([x[i] for i in I])
               


Resuelva el problema 

In [ ]:
m.solve()

1

In [ ]:
print(plp.value(m.objective))

69.0


Modifique la función objetivo del problema.

## Análisis
El resultado del modelo de optimización muestra que se cubren 69 (de 100) barrios. Veamos la solución que logra ese resultado óptimo.

## Plan de ubicación de ambulancias
Este plan determina en qué ubicaciones poner las ambulancias.

In [ ]:
# display optimal values of decision variables using pulp notation

for j in J:
    if y[j].varValue > 0.5:
        print(y[j].name, "=", y[j].varValue)

Select_2 = 1.0
Select_7 = 1.0
Select_10 = 1.0
Select_16 = 1.0
Select_17 = 1.0


In [ ]:
# Depict the optimal solution in plotly figure, use lines to depict  allocation of customers to facilities

fig = go.Figure()
fig.add_trace(go.Scatter(x=[x for x,y in facilities], y=[y for x,y in facilities], mode='markers', name='Facilities'))
fig.add_trace(go.Scatter(x=[x for x,y in customers], y=[y for x,y in customers], mode='markers', name='Hospitals'))

selected_facilities = [facilities[j] for j in J if y[j].varValue > 0.5]

# set marker size with a fixed value in the same scale as the plot

fig.add_trace(go.Scatter(x=[x for x,y in selected_facilities], y=[y for x,y in selected_facilities], mode='markers', name='Selected facilities', marker=dict(size=250, color='rgba(255, 0, 0, 0.2)')))
fig.update_layout(title='Facilities and hospitals', xaxis_title='x', yaxis_title='y')
# set plot limits
fig.update_xaxes(range=[0, 3])
fig.update_yaxes(range=[0, 3])
fig.show()

### Plan de Envío
Este plan determina el porcentaje de envíos que se enviarán desde cada instalación construida a cada cliente.

In [ ]:

for j in J:
    if (abs(y[j].varValue) > 1e-6):
        print('Ubicación', j, 'Cubre los clientes:')
        print([i for i in I if (abs(beta[i,j]) ==1)])



Ubicación 2 Cubre los clientes:
[22, 23, 24, 31, 32, 33, 34, 41, 42, 43, 44, 51, 52, 53, 54]
Ubicación 7 Cubre los clientes:
[5, 6, 7, 8, 15, 16, 17, 18, 25, 26, 27, 28, 35, 36, 37]
Ubicación 10 Cubre los clientes:
[60, 61, 62, 70, 71, 72, 73, 80, 81, 82, 83, 90, 91, 92]
Ubicación 16 Cubre los clientes:
[37, 38, 39, 46, 47, 48, 49, 56, 57, 58, 59, 67, 68, 69, 78]
Ubicación 17 Cubre los clientes:
[44, 45, 53, 54, 55, 56, 63, 64, 65, 66, 73, 74, 75, 76, 84, 85]


##  Conclusión
En este ejemplo, abordamos un problema de ubicación de ambulancias en el que deseamos satisfacer la demanda la mayor cantidad posible de barrios sin sobrepasar la cantidad de ambulancias disponibles. Aprendimos cómo formular el problema como un modelo MIP. 

También aprendimos cómo implementar la formulación del modelo MIP y resolverlo utilizando la API de Python de Cbc Solver. 

##  Referencias
[1] Laporte, Gilbert, Stefan Nickel, and Saldanha da Gama, Francisco. Location Science. Springer, 2015.